Cartar de posible desvio para la union

In [2]:
import sys
sys.path.append('..')

import pandas as pd
import collections
import os

from docxtpl import DocxTemplate

from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import POSTGRES_UTEA

RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)

In [3]:
RUTA_COMPLETA

'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [4]:
path_xlsx_avance = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\AVANCE SEMANAL\AVANCE DE COSECHA V1.xlsx'

In [5]:
xlsx_entregas = pd.read_excel(path_xlsx_avance, sheet_name='DETALLE ENTREGAS')
xlsx_avance = pd.read_excel(path_xlsx_avance, sheet_name = 'AVANCE')

C:\Users\Usuario\anaconda3\envs\utea\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
C:\Users\Usuario\anaconda3\envs\utea\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [7]:
xlsx_entregas = xlsx_entregas.drop(xlsx_entregas.index[-1])
xlsx_entregas = xlsx_entregas.drop(xlsx_entregas.index[-1])
xlsx_entregas['cod_cos'] = xlsx_entregas['cod_cos'].astype(int)
xlsx_entregas['cod_ca'] = xlsx_entregas['cod_ca'].astype(int)
xlsx_entregas['%ENTREGA'] = xlsx_entregas['%ENTREGA'] * 100
xlsx_entregas.head(3)

,inst,cod_cos,cod_ca,nom_ca,TOTAL COMPROMISO,TOTAL ENTREGAS,%ENTREGA
0,1,49,660,AGUILERA ANTELO ALEX ENRIQUE,982.61,627.21,63.831021
1,1,54,3361,ENRIQUEZ ROJAS ALBERTO,1615.39,246.99,15.289806
2,1,55,3460,ESPINOZA MENDOZA SANTIAGO,5288.56,4826.73,91.267377


In [8]:
xlsx_avance.head(3)

,COD COS,COD CA,NOMBRE CAÑERO,COMPROMISO,ENTREGAS,%ENTREGA,AREA,COSECHADO,SIN COSECHAR,SEMILLA,...,PRODUCCION EST,EST TN COSECHA,DIF EST ENTREGA,%DIF EST ENTREGA,TN EN CAMPO,COMPROMISO REST,DIFERENICA,COMPRADOR,DESVIADOR,EST VS COMPRIM
0,544,5334,HUALLPA AUCACHI JUSTINO,2958.93,2248.01,0.759737,37.172376,16.748978,20.423398,NaN,...,1991.766094,922.792097,1325.217903,0.410493,1068.973996,710.92,358.053996,1,0,0.673137
1,602,551,AQUINO PAITA ELIAS,1253.55,1128.61,0.900331,23.142961,10.392392,11.470565,1.280004,...,1106.831999,540.297818,588.312182,0.478729,566.534182,124.94,441.594182,1,0,0.882958
2,564,172,ALVAREZ ALAVE JORGE,1010.36,1252.95,1.240103,15.750806,12.518848,3.231958,NaN,...,748.728442,601.263242,651.686758,0.479878,147.465200,-242.59,390.055200,1,0,0.741051


In [14]:
fecha_inf = '18 de septiembre de 2025'
fecha_form = '18-09-2025'

fecha_img = ' de septiembre de 2025'

cod_grupos = [523,
414,
450,
370,
526,
371
]


In [15]:
def crear_reporte(lista_cod_cos):
    for cod in lista_cod_cos:
        grupo = xlsx_entregas[xlsx_entregas['cod_cos'] == cod].copy()
        
        # convertir datos del grupo a objetos
        list_obj_caneros = []
        for index, row in grupo.iterrows():
            canero = {
            'inst' : row['inst'],
            'cod_ca' : row['cod_ca'],
            'nom_ca' : row['nom_ca'],
            'compromiso' : row['TOTAL COMPROMISO'],
            'entrega' : row['TOTAL ENTREGAS'],
            'entrega_porcen' : row['%ENTREGA']
            }
            list_obj_caneros.append(collections.namedtuple("canero", canero.keys())(*canero.values()))
        # totales de los datos del grupo
        grupo_totales = {
            'compromiso' : grupo['TOTAL COMPROMISO'].sum(),
            'entregas' : grupo['TOTAL ENTREGAS'].sum(),
            'entrega_porcen' : grupo['TOTAL ENTREGAS'].sum() / grupo['TOTAL COMPROMISO'].sum() * 100
        }
        obj_grupo_totales = collections.namedtuple("grupo_totales", grupo_totales.keys())(*grupo_totales.values())
        
        # datos generales de avance de cosecha
        avance = xlsx_avance[xlsx_avance['COD COS'] ==cod]
        avance_totales = {
            'fecha_img' : fecha_img,
            'fecha_informe' : fecha_inf,
            'compromiso' : float(avance['COMPROMISO'].iloc[0]),
            'entrega' : float(avance['ENTREGAS'].iloc[0]),
            'entrega_porcen' : float(avance['%ENTREGA'].iloc[0]) * 100,
            'ha_canha' : float(avance['COSECHADO'].iloc[0] + avance['SIN COSECHAR'].iloc[0]),
            'ha_cosecha' : float(avance['COSECHADO'].iloc[0]),
            'cosecha_porcen' : float(avance['%COSECHA'].iloc[0]) * 100,
            'tch' : float(avance['TCH'].iloc[0])
        }
        # datos generales a objeto
        obj_avance_totales = collections.namedtuple("avance_totales",avance_totales.keys())(*avance_totales.values())
        
        # generar reporte
        doc = DocxTemplate(RUTA_COMPLETA + '/templates/tpl_rpt_avance_posible_desvio.docx')
        context = {
            'insp' : list_obj_caneros,
            'grupo_totales' : obj_grupo_totales,
            'avance' : obj_avance_totales
        }
        doc.render(context)

        #REPROTE POSIBLE COMPRA DE CAÑA
        doc.save(RUTA_COMPLETA + '/informes/' + str(avance['COD CA'].iloc[0]) + '_RPCC_' + fecha_form + '_' + avance['NOMBRE CAÑERO'].iloc[0] + '.docx')
        print('Se generó reporte de grupo:', cod, ' - ', avance['NOMBRE CAÑERO'].iloc[0])

In [16]:
crear_reporte(cod_grupos)

Se generó reporte de grupo: 523  -  PIZARRO BRAVO ROLY ANDRES
Se generó reporte de grupo: 414  -  RIBERA JUSTINIANO JEFFER PAUL
Se generó reporte de grupo: 450  -  CLAROS RODRIGUEZ EMERSON
Se generó reporte de grupo: 370  -  TORREJON MENDOZA EULOGIO
Se generó reporte de grupo: 526  -  AGUILERA PALMA JORGE ENRIQUE
Se generó reporte de grupo: 371  -  REYES VALDIVIA JOSE
